# --- Day 11 Monkey in the Middle ---

https://adventofcode.com/2022/day/11

In [1]:
from collections import defaultdict

## Get Input Data

In [3]:
old = 5
eval('old + 5')

10

In [42]:
def get_data(filename):

    monkey_notes = defaultdict(dict)

    with open(f'../inputs/{filename}.txt') as _file:
        for line in _file:
            line = line.rstrip()
            if line.startswith('Monkey'):
                ID = int(line[-2])  # There're only 8 monkeys in the input file
                # monkey_notes[ID] = {}
            elif line.startswith('  Starting items:'):
                monkey_notes[ID]['items'] = [int(x) for x in line[18:].split(',')]
            elif line.startswith('  Operation:'):
                monkey_notes[ID]['op'] = line[19:]
            elif line.startswith('  Test:'):
                monkey_notes[ID]['test'] = int(line[21:])
            elif line.startswith('    If true:'):
                monkey_notes[ID]['true'] = int(line[-1])
            elif line.startswith('    If false:'):
                monkey_notes[ID]['false'] = int(line[-1])

    return monkey_notes

In [43]:
test_monkey_notes = get_data('test_monkey_notes')
test_monkey_notes

defaultdict(dict,
            {0: {'items': [79, 98],
              'op': 'old * 19',
              'test': 23,
              'true': 2,
              'false': 3},
             1: {'items': [54, 65, 75, 74],
              'op': 'old + 6',
              'test': 19,
              'true': 2,
              'false': 0},
             2: {'items': [79, 60, 97],
              'op': 'old * old',
              'test': 13,
              'true': 1,
              'false': 3},
             3: {'items': [74],
              'op': 'old + 3',
              'test': 17,
              'true': 0,
              'false': 1}})

## Part 1
---

In [70]:
def pass_items_around(monkey_notes):
    """Simulate 20 rounds of monkeys inspecting items in your backpak and thowing them to another monkey.
    
    Parameters
    ----------
    monkey_notes : dict
        Contains details of 'worry level' for each item that a monkey has taken from your backpack.
    
    Returns
    -------
    monkey_business : int
        Measure of the product of the top two most productive monkeys 
        (those who have inspected the most items)
    """
    # Keep track of how many items each monkey has inspected
    inspected_counts = [0] * len(monkey_notes)
    
    for round in range(20):
        for m in range(len(monkey_notes)):
            for item in monkey_notes[m]['items']:
                inspected_counts[m] += 1
                old = item  # Use 'old', because that's what the op says
                new = eval(monkey_notes[m]['op']) // 3
                test = new % monkey_notes[m]['test'] == 0
                if test:
                    next_monkey = monkey_notes[m]['true']
                else:
                    next_monkey = monkey_notes[m]['false']
                # Throw the item to the next monkey
                monkey_notes[next_monkey]['items'].append(new)
            
            # Clear current monkey's items list
            monkey_notes[m]['items'] = []

    # Calculate "monkey business" metric
    top_two = sorted(inspected_counts)[-2:]
    monkey_business = top_two[0] * top_two[1]

    return monkey_business

### Run on Test Data

In [71]:
pass_items_around(get_data('test_monkey_notes')) == 10605

True

### Run on Input Data

In [72]:
pass_items_around(get_data('monkey_notes'))

55458

## Part 2
---

### Run on Test Data

In [ ]:
test_data = {
    
}

In [ ]:
for k, v in test_data.items():
    assert bar(k) == v
print('Woot! Passed test cases!')

### Run on Input Data